In [ ]:
#!pip install import-ipynb
import pandas as pd
import numpy as np
import requests as rq
import import_ipynb
import util as u
from bs4 import BeautifulSoup

In [ ]:
def get_dados_discursos(codigo_parlamentar, ano):
    dataInicio = str(ano) + "0101"
    dataFim = str(ano) + "1231"
    urlDiscursos = "https://legis.senado.leg.br/dadosabertos/senador/"+str(codigo_parlamentar)+"/discursos?dataInicio="+dataInicio+"&dataFim="+dataFim    
    df = u.parse_XML(urlDiscursos,["CodigoPronunciamento","UrlTexto","DataPronunciamento","TextoResumo","TipoUsoPalavra/Sigla","TipoUsoPalavra/Descricao"],"Parlamentar/Pronunciamentos/Pronunciamento")
    df = df.set_index('CodigoPronunciamento')
    df.rename(columns={'TipoUsoPalavra/Sigla': 'TipoSigla', 'TipoUsoPalavra/Descricao': 'TipoDescricao'}, inplace=True)
    df['CodigoParlamentar'] = codigo_parlamentar
    return df

In [ ]:
def get_materias_citadas_discursos(codigoParlamentar, ano):
    df_discursos_URL = get_dados_discursos(codigoParlamentar,ano)
    df_materias_discurso = pd.DataFrame()
    for codigo, row in df_discursos_URL.iterrows():            
        url = row['UrlTexto']
        dataDiscurso = row['DataPronunciamento']
        resumoDiscurso = row['TextoResumo']
        tipoSigla = row['TipoSigla']
        tipoDescricao = row['TipoDescricao']
        siglasAceitas = ['POR','PDI','PEN','PL','DIS']
        if (tipoSigla in siglasAceitas):
            page = rq.get(url)    
            soup = BeautifulSoup(page.text, 'html.parser')    
            for link in soup.find_all('a'):
                href = link.get('href')          
                
                if (href is not None):            
                    texto_pesquisar = 'materia/'
                    indice = href.find(texto_pesquisar)
                    if (indice!=-1):
                        codigo_materia = int(href[indice+len(texto_pesquisar):])                
                        data = [[codigoParlamentar, codigo, url, ano, dataDiscurso,resumoDiscurso, codigo_materia, tipoSigla, tipoDescricao]]
                        df_materia = pd.DataFrame(data,columns=['codigoParlamentar', 'codigoDiscurso','urlDiscurso','anoDiscurso','dataDiscurso','resumoDiscurso','codigoMateria','tipoSigla','tipoDescricao'])
                        #print(df_materia)
                        df_materias_discurso = pd.concat([df_materias_discurso,df_materia])
    return df_materias_discurso

In [ ]:
def get_complementacao_materias_citadas_discursos(codigoParlamentar,ano):
    
    
    df_discursos_URL = get_dados_discursos(codigoParlamentar,ano)    

    #Filtrando os que já existem na lista    
    codigo_discurso_lista = df_discursos['codigoDiscurso'].tolist()    
    lista_strings = [str(valor) for valor in codigo_discurso_lista]
    mascara = ~df_discursos_URL.index.isin(lista_strings)
    df_discursos_URL_filtrado = df_discursos_URL[mascara]

    df_materias_discurso = pd.DataFrame()
    for codigo, row in df_discursos_URL_filtrado.iterrows():            
        url = row['UrlTexto']
        dataDiscurso = row['DataPronunciamento']
        resumoDiscurso = row['TextoResumo']
        tipoSigla = row['TipoSigla']
        tipoDescricao = row['TipoDescricao']
        siglasAceitas = ['POR','PDI','PEN','PL','DIS','APA']
        if (tipoSigla in siglasAceitas):
            page = rq.get(url)    
            soup = BeautifulSoup(page.text, 'html.parser')    
            for link in soup.find_all('a'):
                href = link.get('href')          

                if (href is not None):            
                    texto_pesquisar = 'materia/'
                    indice = href.find(texto_pesquisar)
                    if (indice!=-1):
                        codigo_materia = int(href[indice+len(texto_pesquisar):])                
                        data = [[codigoParlamentar, codigo, url, ano, dataDiscurso,resumoDiscurso, codigo_materia, tipoSigla, tipoDescricao]]
                        df_materia = pd.DataFrame(data,columns=['codigoParlamentar', 'codigoDiscurso','urlDiscurso','anoDiscurso','dataDiscurso','resumoDiscurso','codigoMateria','tipoSigla','tipoDescricao'])
                        #print(df_materia)
                        df_materias_discurso = pd.concat([df_materias_discurso,df_materia])
    return df_materias_discurso

## Carrega dados

In [ ]:
#Quando é a primeira vez
df_senadores = pd.read_csv("df_senadores.csv",sep=",")


df_senadores
df_senadores['PegouDetalhe'] = False
df_discursos = pd.DataFrame()

In [ ]:
#Continuar pegando os discursos
#df_senadores = pd.read_csv("df_senadores_discursos.csv",sep=",")
df_discursos = pd.read_csv("df_discursos.csv",sep=",")

In [ ]:
#Alimentação INICIAL
parla_aux = df_senadores.loc[df_senadores['PegouDetalhe'] == False]
for index, row in parla_aux.head(5).iterrows():            
        codigo_parlamentar = row['CodigoParlamentar']
        anos = [2019, 2020, 2021, 2022,2023]
        for ano in anos:
            df_discursos_senador = get_materias_citadas_discursos(codigo_parlamentar,ano)
            df_discursos = pd.concat([df_discursos,df_discursos_senador])
            df_senadores.at[index, 'PegouDetalhe'] = True   

In [ ]:
#ATUALIZAÇÃO DOS DISCURSOS INICIAL
parla_aux = df_senadores.loc[df_senadores['PegouDetalhe'] == False]
for index, row in parla_aux.iterrows():            
        codigo_parlamentar = row['CodigoParlamentar']
        anos = [2023]
        for ano in anos:
            df_discursos_senador = get_complementacao_materias_citadas_discursos(codigo_parlamentar,ano)
            df_discursos = pd.concat([df_discursos,df_discursos_senador])
            df_senadores.at[index, 'PegouDetalhe'] = True   

In [ ]:
df_senadores.loc[df_senadores['PegouDetalhe'] == True]

In [ ]:
df_senadores.loc[df_senadores['PegouDetalhe'] == False]

In [ ]:
df_discursos

In [ ]:
len(df_discursos)

In [ ]:
# Remove as duplicatas
df_discursos = df_discursos.drop_duplicates(subset=['codigoParlamentar','codigoDiscurso', 'codigoMateria'])

In [ ]:
# Pesquisa pelas duplicatas
df_discursos.groupby(['codigoDiscurso', 'codigoMateria']).filter(lambda x: len(x) > 1)

In [31]:
df_discursos.to_csv('df_discurso.csv', index=False)

# Área de query

In [ ]:
df_discursos

In [ ]:
df_senadores.loc[df_senadores['PegouDetalhe'] == False]

In [ ]:
df_discursos = pd.read_csv('df_discursos.csv')

In [ ]:
df_discursos